In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# need to fix total df because each week should hold agg stats from only prev weeks, not including itself
# keep fixing mismattches with the weeks when making final df

In [6]:
df2 = pd.read_csv("../data/game_stats.csv")
df2.columns

Index(['Unnamed: 0', 'id', 'home.school', 'home.conference', 'home.points',
       'home.fumblesRecovered', 'home.rushingTDs', 'home.puntReturnYards',
       'home.puntReturnTDs', 'home.puntReturns', 'home.passingTDs',
       'home.kickingPoints', 'home.firstDowns', 'home.thirdDownEff',
       'home.fourthDownEff', 'home.totalYards', 'home.netPassingYards',
       'home.completionAttempts', 'home.yardsPerPass', 'home.rushingYards',
       'home.rushingAttempts', 'home.yardsPerRushAttempt',
       'home.totalPenaltiesYards', 'home.turnovers', 'home.fumblesLost',
       'home.interceptions', 'home.possessionTime', 'away.school',
       'away.conference', 'away.points', 'away.fumblesRecovered',
       'away.rushingTDs', 'away.puntReturnYards', 'away.puntReturnTDs',
       'away.puntReturns', 'away.passingTDs', 'away.kickingPoints',
       'away.interceptionYards', 'away.interceptionTDs',
       'away.passesIntercepted', 'away.firstDowns', 'away.thirdDownEff',
       'away.fourthDownEff', 

In [171]:
def process_third(x):
    if(str(x).split('-')[-1] == '0'): return 1
    else: return float(str(x).split('-')[0])/float(str(x).split('-')[1])

stats = df2[['id', 'home.school','home.points', 'home.thirdDownEff','home.completionAttempts', 'home.yardsPerPass','home.yardsPerRushAttempt','home.totalPenaltiesYards','home.turnovers','home.possessionTime','away.school','away.points','away.thirdDownEff','away.completionAttempts', 'away.yardsPerPass','away.yardsPerRushAttempt','away.totalPenaltiesYards','away.turnovers','away.possessionTime','home.rushingAttempts','away.rushingAttempts','Year','Week']]
stats.dropna(inplace = True)
stats.loc[:,'home.pointDiff'] = df2['home.points'] - df2['away.points']
stats.loc[:,'away.pointDiff'] = -stats['home.pointDiff']
stats.loc[:,'home.completionAttempts'] = df2['home.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'away.completionAttempts'] = df2['away.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'home.yardsAllowedPerPlay'] = df2['away.totalYards']/(stats['away.completionAttempts'] + df2['away.rushingAttempts'])
stats.loc[:,'away.yardsAllowedPerPlay'] = df2['home.totalYards']/(stats['home.completionAttempts'] + df2['home.rushingAttempts'])
stats.loc[:,'home.yards_per_play'] = stats['away.yardsAllowedPerPlay']
stats.loc[:,'away.yards_per_play'] = stats['home.yardsAllowedPerPlay']
stats.loc[:,'home.forcedTO'] = df2['away.turnovers']
stats.loc[:,'away.forcedTO'] = df2['home.turnovers']
stats.loc[:,'home.passAttemptsAllowed'] = stats['away.completionAttempts']
stats.loc[:,'away.passAttemptsAllowed'] = stats['home.completionAttempts']
stats.loc[:,'home.yardsAllowedPerRush'] = df2['away.yardsPerRushAttempt']
stats.loc[:,'away.yardsAllowedPerRush'] = df2['home.yardsPerRushAttempt']
stats.loc[:,'home.thirdDownEff'] = stats.loc[:,'home.thirdDownEff'].apply(process_third)
stats.loc[:,'away.thirdDownEff'] = stats.loc[:,'away.thirdDownEff'].apply(process_third)
stats.loc[:,'home.totalPenaltiesYards'] = stats.loc[:,'home.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'away.totalPenaltiesYards'] = stats.loc[:,'away.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'home.possessionTime'] = stats['home.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats.loc[:,'away.possessionTime'] = stats['away.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats['home.win'] = (stats['home.points'] > stats['away.points']).astype(int)
stats['away.win'] = (stats['away.points'] > stats['home.points']).astype(int)
stats.iloc[1239,22] = 10
stats = stats.drop(9284, axis = 0)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [172]:
away_index = [x for x in list(stats.columns) if 'away.' in x] + ['Week', 'Year', 'id']
away = stats[away_index].drop('away.points', axis = 1)
away.columns = [x.split('.')[-1] for x in list(away.columns)]
away['isHome'] = 0
away = away.groupby(['Year', 'school','Week']).mean()

In [173]:
home_index = [x for x in list(stats.columns) if 'home.' in x] + ['Week', 'Year', 'id']
home = stats[home_index].drop('home.points',axis = 1)
home.columns = [x.split('.')[-1] for x in list(home.columns)]
home['isHome'] = 1
home = home.groupby(['Year', 'school','Week']).mean()

In [174]:
total = pd.concat([home,away]).groupby(['Year', 'school','Week']).mean()
total.iloc[:20,:]

thirdDownEff  completionAttempts  yardsPerPass  \
Year school    Week                                                   
2004 Air Force 2         0.555556                18.0           4.9   
               3         0.600000                16.0           9.1   
               4         0.375000                14.0           9.9   
               5         0.523810                16.0          10.6   
               6         0.473684                24.0           9.7   
               7         0.357143                 9.0          10.6   
               9         0.266667                24.0           6.3   
               10        0.538462                21.0           9.0   
               11        0.416667                 8.0          10.5   
               12        0.428571                20.0           6.7   
               13        0.642857                19.0          10.7   
     Akron     2         0.384615                36.0           6.2   
               3         0.416667                44.0           9.1   
               4         0.062500                27.0           2.9   
               5         0.333333                29.0           6.1   
               6         0.181818                26.0           7.4   
               7         0.416667                23.0           8.8   
               8         0.428571                24.0           7.0   
               9         0.461538                33.0           7.5   
               11        0.250000                45.0           7.2   

                     yardsPerRushAttempt  totalPenaltiesYards  turnovers  \
Year school    Week                                                        
2004 Air Force 2                     3.4                  0.0        2.0   
               3                     6.8                  0.0        1.0   
               4                     3.9                  0.0        0.0   
               5                     4.6                  0.0        3.0   
               6                     3.2                  0.0        2.0   
               7                     4.5                  0.0        0.0   
               9                     4.5                  0.0        0.0   
               10                    4.8                  0.0        2.0   
               11                    5.2                  0.0        0.0   
               12                    5.4                  0.0        6.0   
               13                    5.3                  0.0        0.0   
     Akron     2                     2.3                  0.0        3.0   
               3                     3.0                  0.0        3.0   
               4                     0.2                  0.0        2.0   
               5                     3.2                  0.0        1.0   
               6                     1.9                  0.0        1.0   
               7                     6.1                  0.0        0.0   
               8                     4.0                  0.0        1.0   
               9                     4.5                  0.0        3.0   
               11                    2.5                  0.0        2.0   

                     possessionTime  rushingAttempts  pointDiff  \
Year school    Week                                               
2004 Air Force 2             1213.0             53.0      -42.0   
               3             1131.0             52.0       22.0   
               4             1454.0             71.0       17.0   
               5             1273.0             66.0      -14.0   
               6             1253.0             57.0       -3.0   
               7             1144.0             53.0        5.0   
               9             1067.0             47.0      -17.0   
               10             783.0             48.0      -17.0   
               11            1184.0             66.0        9.0   
               12            1090.0             

In [155]:
idx = list(total.index)
n = 1
ids = total['id']
field_status = total['isHome']
total.drop(['id','isHome'], axis = 1, inplace = True)
for i,x in enumerate(idx):
    if(i == 0): continue
    if(x[1] == idx[i-1][1]):
        total.loc[(x[0],x[1],x[2]),:] = total.loc[(x[0],x[1],x[2]),:] + total.loc[(idx[i-1][0],idx[i-1][1],idx[i-1][2]),:]*n
        total.loc[(x[0],x[1],x[2]),:] = total.loc[(x[0],x[1],x[2]),:]/(n+1)
        n += 1
    else: n = 1;
total['id'] = ids
total['isHome'] = field_status
total.iloc[:50,:]

thirdDownEff  completionAttempts  yardsPerPass  \
Year school            Week                                                   
2004 Air Force         2         0.555556           18.000000      4.900000   
                       3         0.577778           17.000000      7.000000   
                       4         0.510185           16.000000      7.966667   
                       5         0.513591           16.000000      8.625000   
                       6         0.505610           17.600000      8.840000   
                       7         0.480865           16.166667      9.133333   
                       9         0.450266           17.285714      8.728571   
                       10        0.461290           17.750000      8.762500   
                       11        0.456332           16.666667      8.955556   
                       12        0.453556           17.000000      8.730000   
                       13        0.470765           17.181818      8.909091   
     Akron             2         0.384615           36.000000      6.200000   
                       3         0.400641           40.000000      7.650000   
                       4         0.287927           35.666667      6.066667   
                       5         0.299279           34.000000      6.075000   
                       6         0.275787           32.400000      6.340000   
                       7         0.299267           30.833333      6.750000   
                       8         0.317739           29.857143      6.785714   
                       9         0.335714           30.250000      6.875000   
                       11        0.326190           31.888889      6.911111   
                       12        0.326904           30.900000      7.080000   
                       13        0.333549           32.000000      7.354545   
     Alabama           2         0.333333           22.000000      9.300000   
                       3         0.458333           22.000000      8.500000   
                       4         0.448413           25.000000      8.166667   
                       5         0.443452           23.250000      6.925000   
                       6         0.354762           23.400000      6.100000   
                       7         0.368552           21.333333      6.333333   
                       8         0.370846           19.571429      6.571429   
                       9         0.363553           20.625000      6.250000   
                       11        0.350936           20.000000      6.433333   
                       12        0.328342           19.500000      6.550000   
                       13        0.317974           20.454545      6.636364   
     Appalachian State 2         1.000000           31.000000      6.500000   
     Arizona           2         0.500000           23.000000      7.400000   
                       3         0.458333           24.000000      7.650000   
                       4         0.366162           21.000000      7.400000   
                       5         0.366726           23.500000      7.025000   
                       7         0.333381           23.800000      6.360000   
                       8         0.322262           25.500000      6.066667   
                       9         0.304796           26.428571      5.914286   
                       10        0.291697           27.875000      5.950000   
                       11        0.283095           27.555556      6.022222   
                       12        0.271452           28.000000      5.940000   
                       14        0.277078           28.363636      6.109091   
     Arizona State     2         0.312500           44.000000      8.100000   
                       3         0.343750           40.000000      8.100000   
                       4         0.439693           42.000000      8.566667   
                       5         0.396436           41.500000      7.200000   


In [156]:
rowlist = []
for gameID in stats['id']:
    print('Processed: ' + str(gameID))
    hometeam = (total[(total['id'] == gameID) & (total['isHome'] == 1)].values)[0]
    awayteam = (total[(total['id'] == gameID) & (total['isHome'] == 0)].values)[0]
    rowlist.append(hometeam + awayteam)
    rowlist.append(awayteam + hometeam)
matchups = pd.DataFrame(rowlist, columns = (list(total.columns) + ['other.' + x for x in list(total.columns)]))

Processed: 242410259
Processed: 242410193
Processed: 242482426
Processed: 242480120
Processed: 242480153
Processed: 242480228
Processed: 242480152
Processed: 242480164
Processed: 242480356
Processed: 242480275
Processed: 242482294
Processed: 242480130
Processed: 242480194
Processed: 242480135
Processed: 242492509
Processed: 242480084
Processed: 242462628
Processed: 242480218
Processed: 242480026
Processed: 242490264
Processed: 242460009
Processed: 242480024
Processed: 242482005
Processed: 242480012
Processed: 242480099
Processed: 242470167
Processed: 242480059
Processed: 242480277
Processed: 242462050
Processed: 242480213
Processed: 242480158
Processed: 242480038
Processed: 242482306
Processed: 242480142
Processed: 242482305
Processed: 242480066
Processed: 242480251
Processed: 242482567
Processed: 242460254
Processed: 242480201
Processed: 242480005
Processed: 242480242
Processed: 242490097
Processed: 242480145
Processed: 242480344
Processed: 242480276
Processed: 242480252
Processed: 24

Processed: 242970068
Processed: 242970087
Processed: 242970195
Processed: 242970276
Processed: 242972649
Processed: 242972132
Processed: 242970228
Processed: 242970245
Processed: 242960036
Processed: 242970150
Processed: 242972572
Processed: 242972226
Processed: 242970344
Processed: 242970154
Processed: 242970062
Processed: 242972628
Processed: 242970135
Processed: 242970194
Processed: 242970213
Processed: 242970201
Processed: 242972306
Processed: 242972348
Processed: 242960097
Processed: 242970193
Processed: 242970152
Processed: 242970142
Processed: 242972509
Processed: 242972440
Processed: 242970167
Processed: 242970249
Processed: 242972711
Processed: 242970275
Processed: 242970024
Processed: 242970204
Processed: 242970221
Processed: 242950277
Processed: 242970041
Processed: 242972641
Processed: 242970254
Processed: 242970030
Processed: 242970238
Processed: 243042426
Processed: 243040120
Processed: 243040154
Processed: 243040153
Processed: 243040228
Processed: 243040130
Processed: 24

Processed: 252602459
Processed: 252600259
Processed: 252600194
Processed: 252600026
Processed: 252600238
Processed: 252600251
Processed: 252600058
Processed: 252602572
Processed: 252600183
Processed: 252582628
Processed: 252600218
Processed: 252602641
Processed: 252600265
Processed: 252672567
Processed: 252690099
Processed: 252670070
Processed: 252670238
Processed: 252672641
Processed: 252650254
Processed: 252672006
Processed: 252670333
Processed: 252670204
Processed: 252672032
Processed: 252670349
Processed: 252670002
Processed: 252670252
Processed: 252640068
Processed: 252670228
Processed: 252660166
Processed: 252672117
Processed: 252672390
Processed: 252670036
Processed: 252670258
Processed: 252670277
Processed: 252652226
Processed: 252670096
Processed: 252670344
Processed: 252670259
Processed: 252670356
Processed: 252670194
Processed: 252672306
Processed: 252670195
Processed: 252670058
Processed: 252672116
Processed: 252670154
Processed: 252670202
Processed: 252670135
Processed: 25

Processed: 253232641
Processed: 253230239
Processed: 253230252
Processed: 253232439
Processed: 253230021
Processed: 253230167
Processed: 253230023
Processed: 253230328
Processed: 253230152
Processed: 253230249
Processed: 253230068
Processed: 253232229
Processed: 253230120
Processed: 253230058
Processed: 253230242
Processed: 253232572
Processed: 253230276
Processed: 253232655
Processed: 253230061
Processed: 253232633
Processed: 253232579
Processed: 253230002
Processed: 253230145
Processed: 253230008
Processed: 253190193
Processed: 253192006
Processed: 253202649
Processed: 253232050
Processed: 253232084
Processed: 253230349
Processed: 253230258
Processed: 253232390
Processed: 253230153
Processed: 253290062
Processed: 253290009
Processed: 253300024
Processed: 253300097
Processed: 253280277
Processed: 253300164
Processed: 253290038
Processed: 253302305
Processed: 253290245
Processed: 253300201
Processed: 253302440
Processed: 253300023
Processed: 253302653
Processed: 253300249
Processed: 25

Processed: 262800096
Processed: 262800248
Processed: 262800249
Processed: 262800204
Processed: 262800058
Processed: 262800005
Processed: 262802649
Processed: 262802032
Processed: 262800333
Processed: 262800167
Processed: 262800328
Processed: 262802638
Processed: 262810193
Processed: 262872426
Processed: 262870127
Processed: 262870356
Processed: 262870275
Processed: 262870084
Processed: 262870213
Processed: 262870077
Processed: 262870030
Processed: 262872483
Processed: 262870264
Processed: 262870024
Processed: 262870265
Processed: 262870277
Processed: 262862116
Processed: 262872306
Processed: 262870038
Processed: 262870245
Processed: 262872305
Processed: 262870201
Processed: 262870251
Processed: 262872751
Processed: 262852005
Processed: 262872439
Processed: 262870278
Processed: 262870023
Processed: 262872348
Processed: 262880166
Processed: 262870041
Processed: 262852226
Processed: 262872390
Processed: 262870258
Processed: 262870097
Processed: 262870153
Processed: 262870235
Processed: 26

...

Processed: 400548106
Processed: 400547659
Processed: 400547658
Processed: 400548021
Processed: 400547809
Processed: 400548194
Processed: 400548267
Processed: 400548198
Processed: 400548195
Processed: 400548266
Processed: 400548197
Processed: 400560174
Processed: 400547789
Processed: 400547742
Processed: 400548414
Processed: 400548387
Processed: 400547858
Processed: 400547670
Processed: 400548386
Processed: 400548203
Processed: 400548115
Processed: 400547668
Processed: 400548032
Processed: 400548112
Processed: 400548111
Processed: 400548113
Processed: 400548028
Processed: 400547804
Processed: 400547667
Processed: 400548268
Processed: 400547962
Processed: 400547987
Processed: 400547907
Processed: 400547918
Processed: 400548027
Processed: 400547946
Processed: 400548384
Processed: 400547822
Processed: 400547671
Processed: 400547669
Processed: 400548030
Processed: 400548114
Processed: 400547832
Processed: 400547782
Processed: 400547857
Processed: 400547996
Processed: 400548196
Processed: 40

Processed: 400547712
Processed: 400547711
Processed: 400547713
Processed: 400548447
Processed: 400547980
Processed: 400548337
Processed: 400548339
Processed: 400548340
Processed: 400548338
Processed: 400548336
Processed: 400548156
Processed: 400548154
Processed: 400548151
Processed: 400548152
Processed: 400548155
Processed: 400548448
Processed: 400548445
Processed: 400547770
Processed: 400547748
Processed: 400547807
Processed: 400547759
Processed: 400547800
Processed: 400548074
Processed: 400548075
Processed: 400548077
Processed: 400547891
Processed: 400547890
Processed: 400547892
Processed: 400548242
Processed: 400548309
Processed: 400548307
Processed: 400548306
Processed: 400548308
Processed: 400548341
Processed: 400548449
Processed: 400548073
Processed: 400548076
Processed: 400548241
Processed: 400548239
Processed: 400548238
Processed: 400548240
Processed: 400547965
Processed: 400547940
Processed: 400547925
Processed: 400547974
Processed: 400547913
Processed: 400548082
Processed: 40

Processed: 400756937
Processed: 400763521
Processed: 400763518
Processed: 400763608
Processed: 400763426
Processed: 400787246
Processed: 400763422
Processed: 400763609
Processed: 400757038
Processed: 400756932
Processed: 400756935
Processed: 400756938
Processed: 400763423
Processed: 400763522
Processed: 400603848
Processed: 400764871
Processed: 400763526
Processed: 400763520
Processed: 400603849
Processed: 400603845
Processed: 400763519
Processed: 400756933
Processed: 400757035
Processed: 400756931
Processed: 400764870
Processed: 400757043
Processed: 400757041
Processed: 400787250
Processed: 400787248
Processed: 400763619
Processed: 400756940
Processed: 400787252
Processed: 400603885
Processed: 400603880
Processed: 400757042
Processed: 400603881
Processed: 400787447
Processed: 400603878
Processed: 400787445
Processed: 400787444
Processed: 400787249
Processed: 400787251
Processed: 400763615
Processed: 400603882
Processed: 400603884
Processed: 400763618
Processed: 400763528
Processed: 40

Processed: 400787282
Processed: 400787285
Processed: 400787283
Processed: 400787284
Processed: 400787473
Processed: 400763564
Processed: 400763567
Processed: 400763569
Processed: 400763565
Processed: 400763566
Processed: 400763568
Processed: 400764910
Processed: 400764907
Processed: 400764908
Processed: 400764909
Processed: 400787359
Processed: 400787476
Processed: 400603923
Processed: 400756986
Processed: 400603925
Processed: 400603924
Processed: 400603927
Processed: 400603929
Processed: 400603926
Processed: 400603930
Processed: 400603921
Processed: 400603928
Processed: 400603922
Processed: 400787355
Processed: 400787357
Processed: 400787360
Processed: 400787356
Processed: 400787358
Processed: 400787474
Processed: 400756989
Processed: 400756987
Processed: 400756993
Processed: 400756992
Processed: 400756990
Processed: 400756988
Processed: 400763657
Processed: 400763658
Processed: 400760499
Processed: 400763462
Processed: 400763460
Processed: 400763461
Processed: 400763459
Processed: 40

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
400869090 away
matchups.head()